In [ ]:
# default_exp get

# get

> API details.

In [ ]:
#export
from awsSchema.apigateway import Response, Event

In [ ]:
#export
def get(event, *args):
  return Response.returnSuccess(body = {'thisisthegetfunction': 'hellooo'})

In [ ]:
get({})

{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}